### 2021年春 模式识别与机器学习 项目 心跳信号分类预测
### 山东大学 计算机科学与技术学院 智能19 李阳

In [33]:
# 包含所需要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split

#### 查看数据信息

In [34]:
train_data=pd.read_csv('./train.csv',index_col=0) # 训练集
testA_data=pd.read_csv('./testA.csv',index_col=0) # 测试集
print(train_data.shape)
train_data.head().append(train_data.tail()) # 查看数据的行列信息

(100000, 2)


,heartbeat_signals,label
id,,
0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0
99995,"1.0,0.677705342021188,0.22239242747868546,0.25...",0.0
99996,"0.9268571578157265,0.9063471198026871,0.636993...",2.0
99997,"0.9258351628306013,0.5873839035878395,0.633226...",3.0
99998,"1.0,0.9947621698382489,0.8297017704865509,0.45...",2.0


In [36]:
train_data.info # 数据类型

<bound method DataFrame.info of                                        heartbeat_signals  label
id                                                             
0      0.9912297987616655,0.9435330436439665,0.764677...    0.0
1      0.9714822034884503,0.9289687459588268,0.572932...    0.0
2      1.0,0.9591487564065292,0.7013782792997189,0.23...    2.0
3      0.9757952826275774,0.9340884687738161,0.659636...    0.0
4      0.0,0.055816398940721094,0.26129357194994196,0...    2.0
...                                                  ...    ...
99995  1.0,0.677705342021188,0.22239242747868546,0.25...    0.0
99996  0.9268571578157265,0.9063471198026871,0.636993...    2.0
99997  0.9258351628306013,0.5873839035878395,0.633226...    3.0
99998  1.0,0.9947621698382489,0.8297017704865509,0.45...    2.0
99999  0.9259994004527861,0.916476635326053,0.4042900...    0.0

[100000 rows x 2 columns]>

In [37]:
train_data.isnull().sum() # 检查是否有缺省值

heartbeat_signals    0
label                0
dtype: int64

In [38]:
train_data['label'].value_counts() # 查看预测值的分布情况

0.0    64327
3.0    17912
2.0    14199
1.0     3562
Name: label, dtype: int64

#### 数据处理